In [1]:
# importing necessary libraries
import sys
import numpy
import pandas
import random
import logging
import warnings
import threading
import mlflow.sklearn
from itertools import cycle, islice
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [3]:
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(experiment_name='Wine Regression')
tags = {
    "team": "Cold Red Wine",
    "dataset": "Wine",
    "release.version": "2.2.2"
}

In [4]:
def eval_metrices(actual, pred):
    root_mean_square_error = numpy.sqrt(mean_squared_error(actual, pred))
    mean_abs_err = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return root_mean_square_error, mean_abs_err, r2

In [5]:
thread_lock = threading.Lock()

def mlflow_func(alpha, l1_ratio, train_x, train_y, test_x, test_y, run_name):
    with mlflow.start_run(run_name=run_name):
        mlflow.set_tags(tags)
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrices(test_y, predicted_qualities)
        
        with thread_lock:
            print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
            print("  RMSE: %s" % rmse)
            print("  MAE: %s" % mae)
            print("  R2: %s" % r2)
            mlflow.log_param("alpha", alpha)
            mlflow.log_param("l1_ratio", l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            mlflow.sklearn.log_model(lr, "model")
            mlflow.log_artifact(local_path='./demo.ipynb', artifact_path='code')

In [6]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    # Read the wine-quality csv file
    run_nams = ['Apples',        'Apricots',     'Asparagus', 
                'Avocados',      'Cherries',     'Basil', 
                'Beets',         'Blackberries', 'Blueberries', 
                'Boysenberries', 'Broccoli',     'Cabbage', 
                'Carrots',       'Collards',     'Corn', 
                'Cucumber',      'Eggplants']

    csv_file_path = "./winequality-red.csv"
    try:
        data = pandas.read_csv(csv_file_path, sep=";")
    except Exception as e:
        logger.exception("Unable to read the CSV file. Error: %s", e)
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    iterations = 5
    # Generate unique run names for each thread
    unique_run_names = list(islice(cycle(run_nams), iterations))

    # Execute threads
    threads = []
    for i, run_name in enumerate(unique_run_names):
        alpha = round(random.random(), 1) # Default value
        if len(sys.argv) > 1:
            try:
                alpha = float(sys.argv[1])
            except ValueError:
                print("Invalid float value provided. Using default value.")
        l1_ratio = round(random.random(), 1)  # Default value
        if len(sys.argv) > 2:
            try:
                l1_ratio = float(sys.argv[2])
            except ValueError:
                print("Invalid float value provided for l1_ratio. Using default value.")
        thread = threading.Thread(
            target=mlflow_func,
            args=(alpha, l1_ratio, train_x, train_y, test_x, test_y, unique_run_names[i]),
        )
        thread.start()
        threads.append(thread)

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided for l1_ratio. Using default value.
Invalid float value provided. Using default value.
Invalid float value provided 

Elasticnet model (alpha=0.300000, l1_ratio=1.000000):
  RMSE: 0.7647156701995642
  MAE: 0.6133811581446489
  R2: 0.10800784586826007
Elasticnet model (alpha=0.000000, l1_ratio=0.400000):
  RMSE: 0.6607977679888831
  MAE: 0.5096263864173072
  R2: 0.3339632547603878
Elasticnet model (alpha=0.000000, l1_ratio=0.200000):
  RMSE: 0.6607977679888831
  MAE: 0.5096263864173072
  R2: 0.3339632547603878
Elasticnet model (alpha=0.700000, l1_ratio=0.000000):
  RMSE: 0.7121219644418415
  MAE: 0.556177070878351
  R2: 0.22648308078019008
Elasticnet model (alpha=0.700000, l1_ratio=0.800000):
  RMSE: 0.7973699637458528
  MAE: 0.6500816735091121
  R2: 0.030203082544138837
Elasticnet model (alpha=0.500000, l1_ratio=1.000000):
  RMSE: 0.7973480369556909
  MAE: 0.6496970912822124
  R2: 0.030256418491315107
Elasticnet model (alpha=0.300000, l1_ratio=0.700000):
  RMSE: 0.7421281332052915
  MAE: 0.5866333129171762
  R2: 0.15992348063640827
Elasticnet model (alpha=0.200000, l1_ratio=0.900000):
  RMSE: 0.733652